In [106]:
# Suzan Iloglu, May 21,2020
# Import packages
import csv
import gurobipy as gp
from itertools import product
import geopandas as gpd
import pandas as pd
import numpy as np
import math
import time
import requests
import io
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
pd.options.display.max_columns =200
from IPython.display import Image


# MAPPING THE NEW POLITICS OF CARE: COMMUNITY HEALTH WORKERS
The project presents multiple options for how individual workers in such a Community Health Corps might be distributed within each state. It shows that what you choose to prioritize greatly impacts where care would be sent. We can define communities in greatest need in many ways: we can think about our current crisis and send people to where the COVID19 pandemic rages most fiercely; we can think of long term measures of social and economic inequality embedded in metrics like the Centers for Disease Control and Prevention’s Social Vulnerability Index; we can focus on the places with too many people dying too young and use the County Health Rankings Years-of-Potential-Life-Lost measure; we can think of joblessness and how the pandemic has thrown many into unemployment and target our resources in this way. 

The followings are our options to choose to define vulnerability:


- SOCIAL VULNERABILITY INDEX
- MEDICAID 
- UNEMPLOYMENT
- YEARS OF POTENTIAL LIFE LOST
- TOTAL COVID CASES
- COVID CASES BY POPULATION
- COVID DEATHS BY POPULATION

We will start with Social Vulnerability Index (SVI) from CDC website.

### I. Importing SVI data which includes the variables for calculating county SVI for each state
The CDC uses both a USA-wide and a state by state SVI scores. For our project given that funding is likely going to be managed at a state level, using a state by state SVI scores makes the most sense and will be most sensitive to regional socioeconomic differences. Even though the CDC SVI scores are calculated using percentile rankings, the data sets include raw data estimates for each variables. The following table shows the variablaes used in the method of calculating SVI scores. 




      American Community Survey (ACS), 2014-2018 (5-year) data for the following estimates:
<img src="Data/img/SVI_comp.png" width="500">


Note: Full documentation for 2018 data is available <a href="https://svi.cdc.gov/data-and-tools-download.html">here</a> 
This part of the code shows preliminary mapping of <a href = "https://svi.cdc.gov/">the CDC's Social Vulnerability Index</a>.

Later in the notebook, we will provide the formula to create the SVI value we use in our project. First, we import the data for the US mainland and Puerto Rico.

In [107]:
## import svi data downloaded from CDC website as cited above

## 48 state SVI scores by county
svi_counties_mainland = gpd.read_file("Data/SVI2018_US_COUNTY/SVI2018_US_county.shp")

## Puerto Rico SVI scores by county
svi_counties_puerto_rico = gpd.read_file("Data/PuertoRico_COUNTY/SVI2018_PuertoRico_county.shp")

## Merge 48 states and Puerto Rico SVI 
svi_counties = pd.concat([svi_counties_mainland,svi_counties_puerto_rico ], sort = False)


In [108]:
## Replacing -999 values with 0 for calculations
svi_county = svi_counties.fillna(0)
svi_county  = svi_county.replace(-999, 0)
svi_county['FIPS'] = svi_county['FIPS'].astype(int)

In [109]:
## Create the list for State
State = svi_county.STATE.unique().tolist()

In [110]:
# Create a seperate dictionary for the variables to calculate SVI

# Persons below poverty estimate, 2014-2018 ACS
E_POV = dict(zip(svi_county.FIPS, svi_county.E_POV))

# Civilian (age 16+) unemployed estimate, 2014-2018 ACS
E_UNEMP = dict(zip(svi_county.FIPS, svi_county.E_UNEMP))

# Per capita income estimate, 2014-2018 ACS
E_PCI = dict(zip(svi_county.FIPS, svi_county.E_PCI))

# Persons (age 25+) with no high school diploma estimate, 2014-2018 ACS
E_NOHSDP = dict(zip(svi_county.FIPS, svi_county.E_NOHSDP))

# Persons aged 65 and older estimate
E_AGE65 = dict(zip(svi_county.FIPS, svi_county.E_AGE65))

# Persons aged 17 and younger estimate
E_AGE17 = dict(zip(svi_county.FIPS, svi_county.E_AGE17))

# Population with a disability estimate
E_DISABL = dict(zip(svi_county.FIPS, svi_county.E_DISABL))

# Single parent households with children under 18 estimate
E_SNGPNT = dict(zip(svi_county.FIPS, svi_county.E_SNGPNT))

# Minority (all persons except white, nonHispanic) estimate, 2014-2018 ACS
E_MINRTY = dict(zip(svi_county.FIPS, svi_county.E_MINRTY))

# Persons (age 5+) who speak English "less than well" estimate, 2014-2018 ACS
E_LIMENG = dict(zip(svi_county.FIPS, svi_county.E_LIMENG))

# Housing in structures with 10 or more units estimate, 2014-2018 ACS
E_MUNIT = dict(zip(svi_county.FIPS, svi_county.E_MUNIT))

# Mobile homes estimate MOE, 2014-2018 ACS
E_MOBILE = dict(zip(svi_county.FIPS, svi_county.E_MOBILE))

# At household level (occupied housing units), more people than rooms estimate, 2014-2018 ACS
E_CROWD = dict(zip(svi_county.FIPS, svi_county.E_CROWD))

# Households with no vehicle available estimate, 2014-2018 ACS
E_NOVEH = dict(zip(svi_county.FIPS, svi_county.E_NOVEH))

# Persons in institutionalized group quarters estimate, 2014-2018 ACS
E_GROUPQ = dict(zip(svi_county.FIPS, svi_county.E_GROUPQ))

# Percentage of persons below poverty estimate
E_POV = dict(zip(svi_county.FIPS, svi_county.E_POV))

# Medicaid 
Medicaid is a means-tested health insurance program for low-income children, pregnant women, adults, seniors, and people with disabilities. Medicaid is jointly funded by federal and state governments and managed by states within federal standards and a wide range of state options. <a href="https://data.medicaid.gov/Enrollment/State-Medicaid-and-CHIP-Applications-Eligibility-D/n5ce-jxme"> Data Source for Medicaid Enrollment </a> 

In [111]:

import sodapy
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.medicaid.gov", None)


# Returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("83yt-67it", limit=4000)


# Read the medicaid demand data
#df_mm = pd.read_csv("Data/2020_06_Preliminary_applications__eligibility_determinations__and_enrollment_data.csv")

# Convert to pandas DataFrame
df_mm = pd.DataFrame.from_records(results)
df_mm.head(5)
df_mm.columns


Index(['applications_for_financial_assistance_submitted_to_the_state_based_marketplace',
       'applications_for_financial_assistance_submitted_to_the_state_based_marketplace_footnotes',
       'final_report', 'geocoded_column',
       'individuals_determined_eligible_for_chip_at_application',
       'individuals_determined_eligible_for_chip_at_application_footnotes',
       'individuals_determined_eligible_for_medicaid_at_application',
       'individuals_determined_eligible_for_medicaid_at_application_footnotes',
       'latitude', 'longitude', 'medicaid_and_chip_child_enrollment',
       'medicaid_and_chip_child_enrollment_footnotes',
       'new_applications_submitted_to_medicaid_and_chip_agencies',
       'new_applications_submitted_to_medicaid_and_chip_agencies_footnotes',
       'preliminary_updated', 'report_date', 'state_abbreviation',
       'state_expanded_medicaid', 'state_name',
       'total_applications_for_financial_assistance_submitted_at_state_level',
       'total_a

In [112]:
df_mm['State Name'] = df_mm['state_name'].str.upper() 

In [113]:
Medicaid_state = dict(zip(df_mm['State Name'], df_mm['total_medicaid_and_chip_enrollment']))
Medicaid_state['PUERTO RICO'] = 1622194
print (Medicaid_state)

{'ALABAMA': '957116', 'ALASKA': '231145', 'ARIZONA': '1839932', 'ARKANSAS': '830467', 'CALIFORNIA': '11847711', 'COLORADO': '1337805', 'CONNECTICUT': '874974', 'DELAWARE': '239009', 'DISTRICT OF COLUMBIA': '250860', 'FLORIDA': '3892552', 'GEORGIA': '1928703', 'HAWAII': '351337', 'IDAHO': '340742', 'ILLINOIS': '2987496', 'INDIANA': '1602976', 'IOWA': '699741', 'KANSAS': '401103', 'KENTUCKY': '1416013', 'LOUISIANA': '1585024', 'MAINE': '232455', 'MARYLAND': '1372695', 'MASSACHUSETTS': '1616404', 'MICHIGAN': '2439425', 'MINNESOTA': '1085778', 'MISSISSIPPI': '632427', 'MISSOURI': '923641', 'MONTANA': '257006', 'NEBRASKA': '254159', 'NEVADA': '685073', 'NEW HAMPSHIRE': '193436', 'NEW JERSEY': '1759653', 'NEW MEXICO': '772102', 'NEW YORK': '6263164', 'NORTH CAROLINA': '1851558', 'NORTH DAKOTA': '96757', 'OHIO': '2788134', 'OKLAHOMA': '797220', 'OREGON': '1053931', 'PENNSYLVANIA': '3069309', 'RHODE ISLAND': '305208', 'SOUTH CAROLINA': '1048276', 'SOUTH DAKOTA': '114059', 'TENNESSEE': '1489536

In [114]:
df_mmm = pd.read_csv("Data/ACSST5Y2019.S2704_data_with_overlays_2020-12-11T225641.csv", header=[1])
df_mmm.head(1)
#df_mmm.dtypes

,id,Geographic Area Name,Estimate!!Total!!Civilian noninstitutionalized population,Margin of Error!!Total!!Civilian noninstitutionalized population,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!Under 19,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!Under 19,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!19 to 64 years,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!19 to 64 years,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!65 years and over,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!65 years and over,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!Under 19,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!Under 19,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!19 to 64 years,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!19 to 64 years,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!65 years and over,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!65 years and over,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!Under 19,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!Under 19,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!19 to 64 years,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!19 to 64 years,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!65 years and over,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!65 years and over,Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Below 138 percent of the poverty threshold,Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Below 138 percent of the poverty threshold,Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!At or above 138 percent of the poverty threshold,Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!At or above 138 percent of the poverty threshold,"Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Worked full-time, year-round (19-64 years)","Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Worked full-time, year-round (19-64 years)",Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Under 6,Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Under 6,Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!6 to 18 years,Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!6 to 18 years,Estimate!!Total!!PUBLIC HEALTH INSURANCE ALON

In [115]:
df_mmm['FIPS'] = df_mmm.id.astype(str).str[9:]
df_mmm.head(5)
df_mmm.FIPS.astype(int)
df_mmm['FIPS'] = pd.to_numeric(df_mmm['FIPS'])

In [116]:
# 2018 ACI data
# ACI_total  = dict(zip(df_mmm['FIPS'], df_mmm["Estimate!!Public Coverage!!Medicaid/means-tested public coverage alone or in combination"]))

#2019 ACI data
ACI_total  = dict(zip(df_mmm['FIPS'], df_mmm["Estimate!!Public Coverage!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination"]))




In [117]:

#df_m = pd.read_csv("Data/Medicaid_Demand.csv")

# Unemployment 
The unemployment rate is calculated by the U.S. Bureau of Labor Statistics as the percentage of the civilian labor force who are without jobs and have actively sought work within the past four weeks. <a href="https://www.bls.gov/lau/laufaq.htm#Q01"> Data Source for Unemployment  </a> 

In [118]:
from io import StringIO
import datetime 
from datetime import date
from dateutil.relativedelta import relativedelta

back = date.today() + relativedelta(months= -3)
three_months_ago = back.strftime('%b-%y') 


url = 'https://www.bls.gov/web/metro/laucntycur14.txt'
s = requests.get(url).text



df_unemp = pd.read_csv(StringIO(s), sep='|',  skiprows=7, skipfooter=6, engine='python', names = ['LAUS Area Code', 'FIPS State', 'FIPS County', 'Area Title', 'Period', 'Civilian Labor Force','Employed','Unemployed_Level','Unemployed_Rate'])

df_unemp['Period'] = df_unemp['Period'].astype(str)

df_unemp['FIPS'] = df_unemp['LAUS Area Code'].str[3:8]

df_unemp = df_unemp[df_unemp['Period'].str.contains(str(three_months_ago))]

df_unemp.head(5)
#df_unemp.dtypes

,LAUS Area Code,FIPS State,FIPS County,Area Title,Period,Civilian Labor Force,Employed,Unemployed_Level,Unemployed_Rate,FIPS
38627,CN0100100000000,1,1,"Autauga County, AL",Oct-20,"25,237","24,120","1,117",4.4,01001
38628,CN0100300000000,1,3,"Baldwin County, AL",Oct-20,"95,828","91,145","4,683",4.9,01003
38629,CN0100500000000,1,5,"Barbour County, AL",Oct-20,"9,633","8,887",746,7.7,01005
38630,CN0100700000000,1,7,"Bibb County, AL",Oct-20,"8,607","8,116",491,5.7,01007
38631,CN0100900000000,1,9,"Blount County, AL",Oct-20,"24,646","23,830",816,3.3,01009


In [119]:
#df_unemp[[ 'FIPS', 'Area Title', 'Period','Unemployed_Level','Unemployed_Rate']].to_csv('Data/County_employment.csv', index=False)


In [120]:


df_unemp.replace({'-', 0})

df_unemp['Unemployed_Level'] = df_unemp['Unemployed_Level'].str.replace(',', '')

df_unemp['Unemployed_Rate'] = df_unemp['Unemployed_Rate'].astype(str)

df_unemp['Unemployed_Level'] = df_unemp['Unemployed_Level'].str.strip()

df_unemp['Unemployed_Rate'] = df_unemp['Unemployed_Rate'].str.strip()

df_unemp['FIPS'] = pd.to_numeric(df_unemp['FIPS'])

df_unemp['Unemployed_Level'] = pd.to_numeric(df_unemp['Unemployed_Level'])

df_unemp['Unemployed_Rate'] = pd.to_numeric(df_unemp['Unemployed_Rate'])




# Fill NA with 0
#df_unemp = df_unemp.fillna(0)
df_unemp.tail(5)
#df_unemp.dtypes


,LAUS Area Code,FIPS State,FIPS County,Area Title,Period,Civilian Labor Force,Employed,Unemployed_Level,Unemployed_Rate,FIPS
41841,CN7214500000000,72,145,"Vega Baja Municipio, PR",Oct-20,"12,548","11,152",1396,11.1,72145
41842,CN7214700000000,72,147,"Vieques Municipio, PR",Oct-20,"2,389","2,135",254,10.6,72147
41843,CN7214900000000,72,149,"Villalba Municipio, PR",Oct-20,"6,563","5,929",634,9.7,72149
41844,CN7215100000000,72,151,"Yabucoa Municipio, PR",Oct-20,"7,959","7,172",787,9.9,72151
41845,CN7215300000000,72,153,"Yauco Municipio, PR",Oct-20,"9,190","8,260",930,10.1,72153


In [121]:
 
## Read the Unemployment data using cvs data 
#df_unemp = pd.read_csv("Data/Unemployment.csv")

# Fill NA with 0
#df_unemp = df_unemp.fillna(0)

#df_unemp.head(5)

# Years of Potential Life Lost (YPLL)

Years of Potential Life Lost (YPLL) measures the rate of premature deaths by region. YPLL is calculated as the sum of the estimated number of years that individuals would have lived if they had not died before the age of 75 per 100,000 people. <a href="https://www.countyhealthrankings.org/sites/default/files/media/document/2020%20County%20Health%20Rankings%20Data%20-%20v2.xlsx"> Data Source for YPLL.  </a> More information about YPLL can be dounf in this <a href="https://www.countyhealthrankings.org/explore-health-rankings/measures-data-sources/county-health-rankings-model/health-outcomes/length-of-life/premature-death-ypll"> link. </a> 


In [122]:
# Read the YPLL data
df_y = pd.read_csv("Data/YPLL.csv")

# Fill NA with the mean of the data
df_y = df_y.fillna(df_y.mean())


# Population

In [123]:
# Read the Population data
df_pop = pd.read_csv("Data/County_pop_2019.csv")

# Fill NA with 0
df_pop = df_pop.fillna(0)


In [124]:
# Create a dictionary for the county and population
population_county = df_pop.set_index('FIPS')['pop'].to_dict()

# Create a dictionary for the county and YPLL
YPLL = dict(zip(df_y.FIPS, df_y.YPLL))

# Create a dictionary for the county and Unemployment
Unemployment = dict(zip(df_unemp.FIPS, df_unemp.Unemployed_Level))


# Create a dictionary for the county and Community Health Workers (CHW) demand
# Note that we assume a CHW can serve 55 Medicaid patient so the demand for CHW will be

#Medicaid_demand = dict(zip(df_m.FIPS, df_m.Med_Demand))


In [125]:
#for m in Medicaid_demand:
#    print (m, Medicaid_demand[m])

# COVID-19 Cases & COVID-19 Cases per Capita

What are COVID-19 Cases and COVID-19 Cases per Capita?

COVID-19 cases is an absolute metric of the total number of COVID-19 cases in a county over the last fourteen days.  COVID-19 cases per 100,000 is a relative metric calculated by dividing the number of COVID-19 cases by the estimated county population and multiplying by 100,000.  Cases include both confirmed cases, based on viral testing, and probable cases, based on specific criteria for symptoms and epidemiological exposure. We use NY Times Covid data. 


In [126]:
#### Data with the most recent date in NY Times dataset:

today = time.strftime('%Y-%m-%d')
covid_data_update_date = today#'2020-07-21'#today #or enter a specific date such as '2020-07-06'


In [127]:
## 14 day period defined
data_date_dt = pd.to_datetime(covid_data_update_date,infer_datetime_format = True)

N = 14

date_N_days_ago = data_date_dt - timedelta(days = N)

date_N1_days_ago = data_date_dt - timedelta(days = N+1)

In [128]:

# URL for mainland US data
url = "http://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv"
s = requests.get(url).content
covid = pd.read_csv(io.StringIO(s.decode('utf-8')))

In [129]:
covid.tail(50)

,date,county,state,fips,cases,deaths
959412,2021-01-23,Ozaukee,Wisconsin,55089.0,8042,74.0
959413,2021-01-23,Pepin,Wisconsin,55091.0,789,7.0
959414,2021-01-23,Pierce,Wisconsin,55093.0,4030,37.0
959415,2021-01-23,Polk,Wisconsin,55095.0,3526,40.0
959416,2021-01-23,Portage,Wisconsin,55097.0,6462,61.0
959417,2021-01-23,Price,Wisconsin,55099.0,1156,7.0
959418,2021-01-23,Racine,Wisconsin,55101.0,21970,313.0
959419,2021-01-23,Richland,Wisconsin,55103.0,1237,13.0
959420,2021-01-23,Rock,Wisconsin,55105.0,14871,146.0
959421,2021-01-23,Rusk,Wisconsin,55107.0,1251,15.0


Note: Since NY data is seperately available, we first read the NY data for all 5 different borough then combine with the rest of US data.

In [130]:
# URL for NY
url = "https://raw.githubusercontent.com/nychealth/coronavirus-data/master/trends/data-by-day.csv"
#"https://raw.githubusercontent.com/nychealth/coronavirus-data/master/data-by-day.csv"

ny = requests.get(url).content
covid_ny = pd.read_csv(io.StringIO(ny.decode('utf-8')))


covid_ny.tail(5)

,date_of_interest,CASE_COUNT,PROBABLE_CASE_COUNT,HOSPITALIZED_COUNT,DEATH_COUNT,PROBABLE_DEATH_COUNT,CASE_COUNT_7DAY_AVG,ALL_CASE_COUNT_7DAY_AVG,HOSP_COUNT_7DAY_AVG,DEATH_COUNT_7DAY_AVG,ALL_DEATH_COUNT_7DAY_AVG,BX_CASE_COUNT,BX_PROBABLE_CASE_COUNT,BX_HOSPITALIZED_COUNT,BX_DEATH_COUNT,BX_PROBABLE_DEATH_COUNT,BX_CASE_COUNT_7DAY_AVG,BX_ALL_CASE_COUNT_7DAY_AVG,BX_HOSPITALIZED_COUNT_7DAY_AVG,BX_DEATH_COUNT_7DAY_AVG,BX_ALL_DEATH_COUNT_7DAY_AVG,BK_CASE_COUNT,BK_PROBABLE_CASE_COUNT,BK_HOSPITALIZED_COUNT,BK_DEATH_COUNT,BK_PROBABLE_DEATH_COUNT,BK_CASE_COUNT_7DAY_AVG,BK_ALL_CASE_COUNT_7DAY_AVG,BK_HOSPITALIZED_COUNT_7DAY_AVG,BK_DEATH_COUNT_7DAY_AVG,BK_ALL_DEATH_COUNT_7DAY_AVG,MN_CASE_COUNT,MN_PROBABLE_CASE_COUNT,MN_HOSPITALIZED_COUNT,MN_DEATH_COUNT,MN_PROBABLE_DEATH_COUNT,MN_CASE_COUNT_7DAY_AVG,MN_ALL_CASE_COUNT_7DAY_AVG,MN_HOSPITALIZED_COUNT_7DAY_AVG,MN_DEATH_COUNT_7DAY_AVG,MN_ALL_DEATH_COUNT_7DAY_AVG,QN_CASE_COUNT,QN_PROBABLE_CASE_COUNT,QN_HOSPITALIZED_COUNT,QN_DEATH_COUNT,QN_PROBABLE_DEATH_COUNT,QN_CASE_COUNT_7DAY_AVG,QN_ALL_CASE_COUNT_7DAY_AVG,QN_HOSPITALIZED_COUNT_7DAY_AVG,QN_DEATH_COUNT_7DAY_AVG,QN_ALL_DEATH_COUNT_7DAY_AVG,SI_CASE_COUNT,SI_PROBABLE_CASE_COUNT,SI_HOSPITALIZED_COUNT,SI_DEATH_COUNT,SI_PROBABLE_DEATH_COUNT,SI_CASE_COUNT_7DAY_AVG,SI_ALL_CASE_COUNT_7DAY_AVG,SI_HOSPITALIZED_COUNT_7DAY_AVG,SI_DEATH_COUNT_7DAY_AVG,SI_ALL_DEATH_COUNT_7DAY_AVG,INCOMPLETE
322,01/16/2021,3282,1036,345,63,7,4477,5705,361,60,67,582,234,76,12,2,836,1091,75,8,9,1017,304,86,19,1,1348,1705,104,17,19,416,144,53,4,0,599,779,51,6,7,981,283,105,26,0,1323,1668,103,24,25,286,71,25,2,0,368,460,28,5,5,11000
323,01/17/2021,2722,1112,324,50,4,4425,5666,364,60,66,465,212,69,8,0,839,1093,77,8,9,849,338,84,20,0,1330,1691,104,18,19,360,171,58,5,0,601,785,54,7,7,817,311,88,15,0,1300,1649,101,23,24,231,79,25,2,0,353,445,28,4,4,11000
324,01/18/2021,4299,1356,388,60,14,4229,5456,369,61,68,703,258,96,15,0,784,1033,80,9,10,1360,429,112,18,0,1283,1645,105,19,20,546,211,56,6,0,562,746,54,7,7,1326,340,98,16,1,1257,1593,101,21,22,364,114,26,5,1,341,435,29,5,5,11000
325,01/19/2021,4522,1343,319,56,23,4165,5401,363,59,69,970,266,67,7,0,786,1037,77,9,10,1287,411,97,21,4,1248,1617,105,18,19,639,217,49,8,0,562,749,54,7,7,1332,361,85,16,1,1240,1576,99,20,21,290,85,21,4,0,327,418,27,5,5,11000
326,01/20/2021,3407,1288,292,34,29,3950,5176,348,55,68,784,229,70,2,1,758,1000,75,9,9,924,375,79,15,1,1172,1541,100,18,19,504,223,45,3,0,540,727,54,6,6,944,381,82,9,0,1170,1509,95,18,19,249,76,16,5,0,309,396,25,4,5,11000


In [131]:
Kings = covid_ny[['date_of_interest', 'BK_CASE_COUNT', 'BK_DEATH_COUNT']]
Kings.rename(columns = {'BK_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'BK_DEATH_COUNT': 'deaths'} , inplace=True)
Kings['county'] = 'Kings'
Kings['state'] = 'New York'
Kings['fips'] = 36047.0
#Kings.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

In [132]:
Bronx = covid_ny[['date_of_interest', 'BX_CASE_COUNT', 'BX_DEATH_COUNT']]
Bronx.rename(columns = {'BX_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'BX_DEATH_COUNT': 'deaths'} , inplace=True)
Bronx['state'] = 'New York'
Bronx['county'] = 'Bronx'
Bronx['fips'] = 36005.0
#Bronx.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

In [133]:
Manhattan = covid_ny[['date_of_interest', 'MN_CASE_COUNT', 'MN_DEATH_COUNT']]
Manhattan.rename(columns = {'MN_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'MN_DEATH_COUNT': 'deaths'} , inplace=True)
Manhattan['state'] = 'New York'
Manhattan['county'] = 'Manhattan'
Manhattan['fips'] = 36061.0
#Manhattan.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

In [134]:
Queens = covid_ny[['date_of_interest', 'QN_CASE_COUNT', 'QN_DEATH_COUNT']]
Queens.rename(columns = {'QN_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'QN_DEATH_COUNT': 'deaths'} , inplace=True)
Queens['state'] = 'New York'
Queens['county'] = 'Queens'
Queens['fips'] = 36081.0
#Queens.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

In [135]:
Richmond = covid_ny[['date_of_interest', 'SI_CASE_COUNT', 'SI_DEATH_COUNT']]
Richmond.rename(columns = {'SI_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'SI_DEATH_COUNT': 'deaths'} , inplace=True)
Richmond['state'] = 'New York'
Richmond['county'] = 'Richmond'
Richmond['fips'] = 36085.0

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

In [136]:
covid.head(5)


,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0.0
1,2020-01-22,Snohomish,Washington,53061.0,1,0.0
2,2020-01-23,Snohomish,Washington,53061.0,1,0.0
3,2020-01-24,Cook,Illinois,17031.0,1,0.0
4,2020-01-24,Snohomish,Washington,53061.0,1,0.0


In [137]:
Bronx.tail(5)

,date,cases,deaths,state,county,fips
322,01/16/2021,582,12,New York,Bronx,36005.0
323,01/17/2021,465,8,New York,Bronx,36005.0
324,01/18/2021,703,15,New York,Bronx,36005.0
325,01/19/2021,970,7,New York,Bronx,36005.0
326,01/20/2021,784,2,New York,Bronx,36005.0


In [138]:
#Kings['deaths'] = Kings.groupby(by=['fips'])['deaths_d'].sum()
#Bronx['deaths'] = Bronx.groupby(by=['fips'])['deaths_d'].sum()
#Manhattan['deaths'] = Manhattan.groupby(by=['fips'])['deaths_d'].sum()
#Queens['deaths'] = Queens.groupby(by=['fips'])['deaths_d'].sum()
#Richmond['deaths'] = Richmond.groupby(by=['fips'])['deaths_d'].sum()

In [139]:
Kings.head(5)

,date,cases,deaths,county,state,fips
0,02/29/2020,0,0,Kings,New York,36047.0
1,03/01/2020,0,0,Kings,New York,36047.0
2,03/02/2020,0,0,Kings,New York,36047.0
3,03/03/2020,0,0,Kings,New York,36047.0
4,03/04/2020,1,0,Kings,New York,36047.0


In [140]:
#Kings = Kings.drop(['deaths_d'], axis=1)
#Bronx = Bronx.drop(['deaths_d'], axis=1)
#Manhattan = Manhattan.drop(['deaths_d'], axis=1)
#Queens = Queens.drop(['deaths_d'], axis=1)
#Richmond = Richmond.drop(['deaths_d'], axis=1)

In [141]:
Kings.tail(5)

,date,cases,deaths,county,state,fips
322,01/16/2021,1017,19,Kings,New York,36047.0
323,01/17/2021,849,20,Kings,New York,36047.0
324,01/18/2021,1360,18,Kings,New York,36047.0
325,01/19/2021,1287,21,Kings,New York,36047.0
326,01/20/2021,924,15,Kings,New York,36047.0


In [142]:
covid['dt'] = pd.to_datetime(covid['date'], infer_datetime_format=True)
Kings['dt'] = pd.to_datetime(Kings['date'], infer_datetime_format=True)
Bronx['dt'] = pd.to_datetime(Bronx['date'], infer_datetime_format=True)
Manhattan['dt'] = pd.to_datetime(Manhattan['date'], infer_datetime_format=True)
Queens['dt'] = pd.to_datetime(Queens['date'], infer_datetime_format=True)
Richmond['dt'] = pd.to_datetime(Richmond['date'], infer_datetime_format=True)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy o

In [143]:

Kings_c = Kings[(Kings['dt']>date_N1_days_ago) & (Kings['dt']<= data_date_dt)].copy()
Queens_c = Queens[(Queens['dt']>date_N1_days_ago) & (Queens['dt']<= data_date_dt)].copy()
Bronx_c = Bronx[(Bronx['dt']>date_N1_days_ago) & (Bronx['dt']<= data_date_dt)].copy()
Manhattan_c = Manhattan[(Manhattan['dt']>date_N1_days_ago) & (Manhattan['dt']<= data_date_dt)].copy()
Richmond_c = Richmond[(Richmond['dt']>date_N1_days_ago) & (Richmond['dt']<= data_date_dt)].copy()

In [144]:
#Kings['cases'] = Kings.loc[(Kings['dt']<= data_date_dt)].groupby(['dt'])['d_cases'].sum()
#Kings['cases'] = Kings.apply(lambda x: x[(Kings['dt']<= data_date_dt)]['d_cases'].sum())

In [145]:
Kings.head(50)

,date,cases,deaths,county,state,fips,dt
0,02/29/2020,0,0,Kings,New York,36047.0,2020-02-29
1,03/01/2020,0,0,Kings,New York,36047.0,2020-03-01
2,03/02/2020,0,0,Kings,New York,36047.0,2020-03-02
3,03/03/2020,0,0,Kings,New York,36047.0,2020-03-03
4,03/04/2020,1,0,Kings,New York,36047.0,2020-03-04
5,03/05/2020,3,0,Kings,New York,36047.0,2020-03-05
6,03/06/2020,1,0,Kings,New York,36047.0,2020-03-06
7,03/07/2020,2,0,Kings,New York,36047.0,2020-03-07
8,03/08/2020,5,0,Kings,New York,36047.0,2020-03-08
9,03/09/2020,16,0,Kings,New York,36047.0,2020-03-09


In [146]:
K_c = Kings_c.groupby(['fips']).sum().reset_index()
Q_c = Queens_c.groupby(['fips']).sum().reset_index()
B_c = Bronx_c.groupby(['fips']).sum().reset_index()
M_c = Manhattan_c.groupby(['fips']).sum().reset_index()
R_c = Richmond_c.groupby(['fips']).sum().reset_index()
K_c.head(19)

,fips,cases,deaths
0,36047.0,13856,195


In [147]:
covid_death = covid[(covid['dt'] >= data_date_dt- timedelta(days = 1))]
Kings_death = Kings.groupby(by=['fips'])['deaths'].sum().reset_index()
Bronx_death = Bronx.groupby(by=['fips'])['deaths'].sum().reset_index()
Manhattan_death = Manhattan.groupby(by=['fips'])['deaths'].sum().reset_index()
Queens_death = Queens.groupby(by=['fips'])['deaths'].sum().reset_index()
Richmond_death = Richmond.groupby(by=['fips'])['deaths'].sum().reset_index()

In [148]:
# We merge the NY data with the rest of the US data
covid = pd.concat([covid, Kings, Bronx, Manhattan, Queens, Richmond], sort = False)


In [149]:
Queens.tail(14)

,date,cases,deaths,state,county,fips,dt
313,01/07/2021,1533,10,New York,Queens,36081.0,2021-01-07
314,01/08/2021,1544,19,New York,Queens,36081.0,2021-01-08
315,01/09/2021,1139,19,New York,Queens,36081.0,2021-01-09
316,01/10/2021,978,19,New York,Queens,36081.0,2021-01-10
317,01/11/2021,1626,31,New York,Queens,36081.0,2021-01-11
318,01/12/2021,1454,21,New York,Queens,36081.0,2021-01-12
319,01/13/2021,1434,27,New York,Queens,36081.0,2021-01-13
320,01/14/2021,1417,24,New York,Queens,36081.0,2021-01-14
321,01/15/2021,1372,18,New York,Queens,36081.0,2021-01-15
322,01/16/2021,981,26,New York,Queens,36081.0,2021-01-16


In [150]:

covid.tail(50)

,date,county,state,fips,cases,deaths,dt
277,12/02/2020,Richmond,New York,36085.0,382,4.0,2020-12-02
278,12/03/2020,Richmond,New York,36085.0,335,3.0,2020-12-03
279,12/04/2020,Richmond,New York,36085.0,327,6.0,2020-12-04
280,12/05/2020,Richmond,New York,36085.0,246,4.0,2020-12-05
281,12/06/2020,Richmond,New York,36085.0,216,2.0,2020-12-06
282,12/07/2020,Richmond,New York,36085.0,369,2.0,2020-12-07
283,12/08/2020,Richmond,New York,36085.0,290,4.0,2020-12-08
284,12/09/2020,Richmond,New York,36085.0,316,5.0,2020-12-09
285,12/10/2020,Richmond,New York,36085.0,276,2.0,2020-12-10
286,12/11/2020,Richmond,New York,36085.0,265,7.0,2020-12-11


In [151]:
print (data_date_dt)

2021-01-24 00:00:00


In [152]:

covid_death = covid_death[['fips', 'deaths']]
covid_death.head(2)

,fips,deaths
956216,1001.0,62.0
956217,1003.0,189.0


In [153]:
Kings_death.head(2)

,fips,deaths
0,36047.0,6453


In [154]:
# We merge the NY data with the rest of the US data
covid_death = pd.concat([covid_death, Kings_death, Bronx_death, Manhattan_death, Queens_death, Richmond_death], sort = False)

covid_death = covid_death.rename(columns={"deaths": "total_deaths"})


# Create a dictionary for the cumulative COVID deaths in each county
County_covid_death = dict(zip(covid_death.fips, covid_death.total_deaths))

covid_death.tail(250)

,fips,total_deaths
959217,51690.0,31.0
959218,51115.0,6.0
959219,51117.0,44.0
959220,51119.0,15.0
959221,51121.0,59.0
959222,51125.0,6.0
959223,51127.0,7.0
959224,51700.0,83.0
959225,51710.0,136.0
959226,51131.0,33.0


In [155]:
covid_death.shape

(3251, 2)

In [156]:
print (County_covid_death)

{1001.0: 62.0, 1003.0: 189.0, 1005.0: 38.0, 1007.0: 48.0, 1009.0: 86.0, 1011.0: 28.0, 1013.0: 60.0, 1015.0: 206.0, 1017.0: 70.0, 1019.0: 31.0, 1021.0: 73.0, 1023.0: 24.0, 1025.0: 36.0, 1027.0: 46.0, 1029.0: 25.0, 1031.0: 64.0, 1033.0: 94.0, 1035.0: 22.0, 1037.0: 18.0, 1039.0: 80.0, 1041.0: 45.0, 1043.0: 124.0, 1045.0: 85.0, 1047.0: 96.0, 1049.0: 107.0, 1051.0: 110.0, 1053.0: 44.0, 1055.0: 181.0, 1057.0: 29.0, 1059.0: 50.0, 1061.0: 47.0, 1063.0: 23.0, 1065.0: 44.0, 1067.0: 25.0, 1069.0: 164.0, 1071.0: 45.0, 1073.0: 1007.0, 1075.0: 22.0, 1077.0: 107.0, 1079.0: 55.0, 1081.0: 107.0, 1083.0: 81.0, 1085.0: 36.0, 1087.0: 37.0, 1089.0: 217.0, 1091.0: 31.0, 1093.0: 61.0, 1095.0: 123.0, 1097.0: 569.0, 1099.0: 21.0, 1101.0: 332.0, 1103.0: 142.0, 1105.0: 18.0, 1107.0: 31.0, 1109.0: 31.0, 1111.0: 36.0, 1113.0: 15.0, 1117.0: 132.0, 1115.0: 130.0, 1119.0: 27.0, 1121.0: 112.0, 1123.0: 109.0, 1125.0: 275.0, 1127.0: 183.0, 1129.0: 27.0, 1131.0: 22.0, 1133.0: 42.0, 2013.0: 0.0, 2016.0: 0.0, 2020.0: 139.0

In [157]:
## subset last last 15 days
covid_last15 = covid[(covid['dt']>date_N1_days_ago) & (covid['dt']<= data_date_dt)].copy()
covid_last15['dt_time_delta'] = covid_last15['dt']-data_date_dt

In [158]:
## calculate new daily cases

## sort values by county and date
covid_last15.sort_values(by=['fips','dt'],inplace=True)
## remove data with 'unknown' counties
covid_last15 = covid_last15[covid_last15['fips'].notnull()].copy()

## calculate daily difference in number of cases
covid_last15['new_cases'] = covid_last15.groupby('fips')['cases'].transform(lambda x: x.diff())
## set negative new cases to zero, this can occuer due to the disperacy in the data

covid_last15.loc[covid_last15.new_cases < 1e-6, 'new_cases'] = 0
covid_last15.sort_index(inplace = True)

In [159]:
covid_last15[covid_last15.county == 'New Haven'].head(50)

,date,county,state,fips,cases,deaths,dt,dt_time_delta,new_cases
914339,2021-01-10,New Haven,Connecticut,9009.0,52278,1627.0,2021-01-10,-14 days,NaN
917584,2021-01-11,New Haven,Connecticut,9009.0,53924,1648.0,2021-01-11,-13 days,1646.0
920829,2021-01-12,New Haven,Connecticut,9009.0,54757,1653.0,2021-01-12,-12 days,833.0
924074,2021-01-13,New Haven,Connecticut,9009.0,55665,1680.0,2021-01-13,-11 days,908.0
927320,2021-01-14,New Haven,Connecticut,9009.0,55943,1685.0,2021-01-14,-10 days,278.0
930565,2021-01-15,New Haven,Connecticut,9009.0,56512,1694.0,2021-01-15,-9 days,569.0
933810,2021-01-16,New Haven,Connecticut,9009.0,56512,1694.0,2021-01-16,-8 days,0.0
937055,2021-01-17,New Haven,Connecticut,9009.0,56512,1694.0,2021-01-17,-7 days,0.0
940300,2021-01-18,New Haven,Connecticut,9009.0,58165,1701.0,2021-01-18,-6 days,1653.0
943545,2021-01-19,New Haven,Connecticut,9009.0,58627,1706.0,2021-01-19,-5 days,462.0


In [160]:
## select just last 14 days now that we have daily new cases with 15th day as baseline
covid_last14 = covid_last15[(covid_last15['dt'] > date_N_days_ago) & (covid_last15['dt'] <= data_date_dt)].copy()


In [161]:
## group by FIPS to get case load and follow up demand values for each county
covid_last14_stats = covid_last14.groupby(['fips'])['new_cases'].sum().reset_index(name ='total_cases')

covid_last14_stats[covid_last14_stats.fips == 9009].head(5)

,fips,total_cases
310,9009.0,7807.0


In [162]:
covid.tail(50)

,date,county,state,fips,cases,deaths,dt
277,12/02/2020,Richmond,New York,36085.0,382,4.0,2020-12-02
278,12/03/2020,Richmond,New York,36085.0,335,3.0,2020-12-03
279,12/04/2020,Richmond,New York,36085.0,327,6.0,2020-12-04
280,12/05/2020,Richmond,New York,36085.0,246,4.0,2020-12-05
281,12/06/2020,Richmond,New York,36085.0,216,2.0,2020-12-06
282,12/07/2020,Richmond,New York,36085.0,369,2.0,2020-12-07
283,12/08/2020,Richmond,New York,36085.0,290,4.0,2020-12-08
284,12/09/2020,Richmond,New York,36085.0,316,5.0,2020-12-09
285,12/10/2020,Richmond,New York,36085.0,276,2.0,2020-12-10
286,12/11/2020,Richmond,New York,36085.0,265,7.0,2020-12-11


In [163]:
## group by FIPS to get case load and follow up demand values for each county
#covid_death = covid.groupby(['fips'])['deaths'].sum().reset_index(name ='total_deaths')
#covid_death.head(250)

In [164]:
## group by FIPS to get case load each month and follow up demand values for each county
covid['month'] = pd.to_datetime(covid['date'])
covid['year'] = pd.to_datetime(covid['date'])


In [165]:
#covid_last14_stats_montly = covid.groupby(['fips', covid.month.dt.month, covid.year.dt.year])['deaths'].sum().reset_index() 

#covid_last14_stats_montly['cumulative_death'] = covid_last14_stats_montly.groupby(['fips'])['deaths'].cumsum(axis = 0) 
#covid_last14_stats_montly = covid_last14_stats_montly.groupby(['fips', covid_last14_stats_montly.month,  covid_last14_stats_montly.year])['deaths'].cumsum()

#covid_last14_stats_montly.head(10)

In [166]:
# adding population information from CDC svi dataset
covid_last14_stats = covid_last14_stats.reset_index()
covid_last14_stats['fips'] = covid_last14_stats['fips'].astype(int)
svi_county['FIPS'] = svi_county['FIPS'].astype(int)
covid_last14_stats = pd.merge(left = covid_last14_stats, right = svi_county[['E_TOTPOP','FIPS', 'STATE']], how = 'right', right_on = 'FIPS', left_on = 'fips' )
covid_last14_stats.fillna(0 , inplace=True)

In [167]:

covid_last14_stats.tail(5)

,index,fips,total_cases,E_TOTPOP,FIPS,STATE
3215,3217.0,72149.0,71.0,22993,72149,PUERTO RICO
3216,3218.0,72151.0,52.0,34149,72151,PUERTO RICO
3217,3219.0,72153.0,34.0,36439,72153,PUERTO RICO
3218,0.0,0.0,0.0,2132,2105,ALASKA
3219,0.0,0.0,0.0,689,2282,ALASKA


In [168]:
now = pd.to_datetime("now")

print (now)
m_now = now.month
print (m_now)

y_now = now.year

2021-01-24 16:35:08.969566
1


In [169]:
#one_month_lag_cumulative_death =  covid_last14_stats_montly[(covid_last14_stats_montly['month'] == m_now-1) & (covid_last14_stats_montly['year'] == y_now)]

# Create a dictionary for the last month COVID deaths in each county
#one_month_lag_death = dict(zip(one_month_lag_cumulative_death.fips, one_month_lag_cumulative_death.cumulative_death))

#one_month_lag_cumulative_death.head(5)


In [170]:
#two_month_lag_cumulative_death =  covid_last14_stats_montly[(covid_last14_stats_montly['month'] == m_now-2) & (covid_last14_stats_montly['year'] == y_now)]

# Create a dictionary for the last month COVID deaths in each county
#two_month_lag_death = dict(zip(two_month_lag_cumulative_death.fips, two_month_lag_cumulative_death.cumulative_death))


#two_month_lag_cumulative_death.head(5)

In [171]:
#three_month_lag_cumulative_death =  covid_last14_stats_montly[(covid_last14_stats_montly['month'] == m_now - 3) & (covid_last14_stats_montly['year'] == y_now)]


# Create a dictionary for the 3 last month COVID deaths in each county
#three_month_lag_death = dict(zip(three_month_lag_cumulative_death.fips, three_month_lag_cumulative_death.cumulative_death))

#three_month_lag_cumulative_death.head(5)

#print (three_month_lag_death)

In [172]:

# Create a dictionary for the states of the given the county FIPS
county_of_states = dict(zip(svi_county.FIPS, svi_county.STATE))

# Create a dictionary for the name of the given the county FIPS
county_name = dict(zip(svi_county.FIPS, svi_county.COUNTY))

# Create the list for county FIPS, we consider counties as analogy to the center for community health workers
location = svi_county.FIPS.tolist() #[k for k in SVI_county] #[9001, 9003, 9005, 9007, 9009, 9011, 9013, 9015]#[k for k in SVI_county]




In [173]:
for j in County_covid_death:
       
    print (j, County_covid_death[j])

1001.0 62.0
1003.0 189.0
1005.0 38.0
1007.0 48.0
1009.0 86.0
1011.0 28.0
1013.0 60.0
1015.0 206.0
1017.0 70.0
1019.0 31.0
1021.0 73.0
1023.0 24.0
1025.0 36.0
1027.0 46.0
1029.0 25.0
1031.0 64.0
1033.0 94.0
1035.0 22.0
1037.0 18.0
1039.0 80.0
1041.0 45.0
1043.0 124.0
1045.0 85.0
1047.0 96.0
1049.0 107.0
1051.0 110.0
1053.0 44.0
1055.0 181.0
1057.0 29.0
1059.0 50.0
1061.0 47.0
1063.0 23.0
1065.0 44.0
1067.0 25.0
1069.0 164.0
1071.0 45.0
1073.0 1007.0
1075.0 22.0
1077.0 107.0
1079.0 55.0
1081.0 107.0
1083.0 81.0
1085.0 36.0
1087.0 37.0
1089.0 217.0
1091.0 31.0
1093.0 61.0
1095.0 123.0
1097.0 569.0
1099.0 21.0
1101.0 332.0
1103.0 142.0
1105.0 18.0
1107.0 31.0
1109.0 31.0
1111.0 36.0
1113.0 15.0
1117.0 132.0
1115.0 130.0
1119.0 27.0
1121.0 112.0
1123.0 109.0
1125.0 275.0
1127.0 183.0
1129.0 27.0
1131.0 22.0
1133.0 42.0
2013.0 0.0
2016.0 0.0
2020.0 139.0
2050.0 14.0
2060.0 0.0
2068.0 0.0
2070.0 1.0
2090.0 25.0
2100.0 0.0
2110.0 3.0
2122.0 18.0
2130.0 1.0
2150.0 5.0
2158.0 3.0
2164.0 0.0
2170

21095.0 35.0
21097.0 2.0
21099.0 15.0
21101.0 53.0
21103.0 5.0
21105.0 7.0
21107.0 115.0
21109.0 23.0
21111.0 770.0
21113.0 48.0
21115.0 16.0
21117.0 93.0
21119.0 14.0
21121.0 10.0
21123.0 6.0
21125.0 17.0
21127.0 8.0
21129.0 18.0
21131.0 2.0
21133.0 5.0
21135.0 29.0
21137.0 29.0
21139.0 15.0
21141.0 45.0
21143.0 11.0
21151.0 47.0
21153.0 1.0
21155.0 20.0
21157.0 32.0
21159.0 11.0
21161.0 24.0
21145.0 48.0
21147.0 18.0
21149.0 24.0
21163.0 5.0
21165.0 4.0
21167.0 11.0
21169.0 12.0
21171.0 33.0
21173.0 20.0
21175.0 0.0
21177.0 50.0
21179.0 23.0
21181.0 5.0
21183.0 37.0
21185.0 63.0
21187.0 7.0
21189.0 10.0
21191.0 1.0
21193.0 24.0
21195.0 39.0
21197.0 0.0
21199.0 73.0
21201.0 9.0
21203.0 12.0
21205.0 5.0
21207.0 36.0
21209.0 24.0
21211.0 31.0
21213.0 17.0
21215.0 5.0
21217.0 28.0
21219.0 17.0
21221.0 6.0
21223.0 3.0
21225.0 10.0
21227.0 95.0
21229.0 16.0
21231.0 36.0
21233.0 15.0
21235.0 14.0
21237.0 4.0
21239.0 4.0
22001.0 174.0
22003.0 77.0
22005.0 135.0
22007.0 30.0
22009.0 103.0
220

47107.0 74.0
47109.0 42.0
47121.0 16.0
47123.0 63.0
47125.0 149.0
47127.0 9.0
47129.0 16.0
47131.0 82.0
47133.0 46.0
47135.0 25.0
47137.0 20.0
47139.0 17.0
47141.0 152.0
47143.0 57.0
47145.0 81.0
47147.0 87.0
47149.0 295.0
47151.0 32.0
47153.0 19.0
47155.0 121.0
47157.0 1182.0
47159.0 30.0
47161.0 20.0
47163.0 220.0
47165.0 232.0
47167.0 69.0
47169.0 15.0
47171.0 45.0
47173.0 25.0
nan 124.0
47175.0 8.0
47177.0 55.0
47179.0 204.0
47181.0 24.0
47183.0 48.0
47185.0 46.0
47187.0 144.0
47189.0 164.0
48001.0 83.0
48003.0 37.0
48005.0 203.0
48007.0 29.0
48009.0 10.0
48011.0 6.0
48013.0 95.0
48015.0 21.0
48017.0 16.0
48019.0 21.0
48021.0 58.0
48023.0 11.0
48025.0 55.0
48027.0 230.0
48029.0 2144.0
48031.0 11.0
48033.0 0.0
48035.0 21.0
48037.0 150.0
48039.0 287.0
48041.0 167.0
48043.0 8.0
48045.0 3.0
48047.0 31.0
48049.0 98.0
48051.0 22.0
48053.0 38.0
48055.0 60.0
48057.0 13.0
48059.0 30.0
48061.0 1260.0
48063.0 33.0
48065.0 12.0
48067.0 63.0
48069.0 25.0
48071.0 31.0
48073.0 101.0
48075.0 13.0


In [174]:
K_c.head()

,fips,cases,deaths
0,36047.0,13856,195


In [175]:
K = dict(zip(K_c.fips, K_c.cases))
Q = dict(zip(Q_c.fips, Q_c.cases))
B = dict(zip(B_c.fips, B_c.cases))
M = dict(zip(M_c.fips, M_c.cases))
R  = dict(zip(R_c.fips, R_c.cases))

In [176]:
print (K)

{36047.0: 13856}


In [177]:
# Create a dictionary for the county and covid cases
covid_cases_county_ny_times = dict(zip(covid_last14_stats.fips, covid_last14_stats.total_cases))
COVID_14days = {}

for j in location:
    if j in covid_cases_county_ny_times:
        COVID_14days[j] = covid_cases_county_ny_times[j] 
        print(j, covid_cases_county_ny_times[j])
        

        
    else:
        COVID_14days[j] = 0
        
        
    if j not in County_covid_death:
        print ('j',j)
        County_covid_death[j] = 0

#print (three_month_lag_death)
#print (two_month_lag_covid_death)
#print (one_month_lag_covid_death)
#for j in location:
#    if j not in three_month_lag_death:        
#        three_month_lag_death[j] = 0
        
#    if j not in two_month_lag_death:        
#        two_month_lag_death[j] = 0
        
#    if j not in one_month_lag_death:        
#        one_month_lag_death[j] = 0
        
    #if j not in County_covid_death:
    #    County_covid_death[j] = 0

35039 394.0
1001 479.0
1009 396.0
1013 99.0
1015 999.0
1017 395.0
1031 462.0
1033 405.0
1039 229.0
1043 555.0
1045 376.0
1051 793.0
1055 959.0
1067 162.0
1069 821.0
1071 380.0
1077 656.0
1079 209.0
1083 721.0
1089 2695.0
1095 566.0
1097 2851.0
1103 867.0
1111 106.0
1113 475.0
1115 595.0
1117 1686.0
1121 649.0
2261 29.0
4021 6312.0
5009 321.0
5011 116.0
5033 760.0
5037 179.0
5045 1177.0
5047 169.0
5051 1022.0
5053 132.0
5063 262.0
5083 215.0
5085 721.0
5087 115.0
5115 699.0
5117 116.0
5121 162.0
5125 1111.0
5131 1681.0
5145 651.0
6007 1159.0
6017 1077.0
6023 463.0
6027 227.0
6061 2066.0
6079 2790.0
6089 1003.0
6093 173.0
8019 23.0
8039 53.0
8047 9.0
8051 188.0
8057 2.0
8065 26.0
8067 196.0
8071 75.0
8075 181.0
8077 1357.0
8083 188.0
8085 447.0
8093 34.0
8099 52.0
8101 357.0
8103 82.0
8119 83.0
8121 14.0
9005 1152.0
9007 1229.0
9011 3250.0
9015 1470.0
10001 1546.0
12019 1585.0
12033 3652.0
12089 895.0
12101 3134.0
12109 2050.0
12113 1768.0
13015 1375.0
13039 361.0
13045 851.0
13047 595.0

55109 454.0
55111 221.0
55115 167.0
55117 551.0
55119 100.0
55121 141.0
55123 75.0
55127 561.0
55131 731.0
55135 308.0
55137 122.0
55141 309.0
56003 69.0
56007 147.0
56009 42.0
56011 12.0
56013 281.0
56015 70.0
56019 65.0
56021 456.0
56025 546.0
56029 286.0
56031 63.0
56033 145.0
56035 20.0
56043 48.0
56045 124.0
1003 1801.0
1019 114.0
1021 190.0
1027 69.0
1041 114.0
1049 495.0
1057 208.0
1061 199.0
1073 5541.0
1075 64.0
1081 1572.0
1093 211.0
1109 286.0
1123 598.0
1125 1350.0
2068 3.0
2090 401.0
2110 57.0
2122 114.0
2195 9.0
2220 18.0
2230 0.0
2275 1.0
4003 1230.0
4019 13523.0
4025 2051.0
5003 170.0
5005 190.0
5007 2521.0
5015 194.0
5019 216.0
5023 179.0
5025 110.0
5029 234.0
5031 866.0
5043 163.0
5055 415.0
5059 316.0
5061 101.0
5071 223.0
5073 57.0
5075 141.0
5081 99.0
5091 273.0
5103 245.0
5105 98.0
5109 83.0
5139 380.0
5143 2490.0
6009 247.0
6033 537.0
6035 297.0
6041 1123.0
6043 24.0
6049 15.0
6057 364.0
6063 45.0
6091 8.0
6105 13.0
6109 263.0
8003 104.0
8005 2437.0
8007 52.0
800

54017 81.0
54019 243.0
54027 157.0
54039 926.0
54061 692.0
54063 103.0
54069 368.0
54071 126.0
54077 344.0
54081 622.0
54093 29.0
55001 98.0
55007 73.0
55013 50.0
55019 154.0
55025 1846.0
55029 170.0
55033 258.0
55037 14.0
55051 39.0
55059 1101.0
55063 651.0
55085 159.0
55099 102.0
55113 92.0
55125 154.0
55129 94.0
55133 2374.0
55139 854.0
56001 111.0
56005 206.0
56017 76.0
56023 65.0
56027 5.0
56037 222.0
56041 179.0
1007 172.0
1037 55.0
1053 373.0
1101 1809.0
1107 109.0
1127 294.0
1129 97.0
2020 931.0
2100 1.0
2130 14.0
4005 1791.0
4011 46.0
5001 219.0
5013 47.0
5021 108.0
5027 151.0
5119 3623.0
5129 50.0
5135 104.0
5141 160.0
5149 240.0
6005 233.0
6015 57.0
6051 146.0
6055 1234.0
6087 1970.0
6097 3266.0
6103 542.0
6111 12416.0
8033 6.0
8037 480.0
8045 327.0
8053 4.0
8055 6.0
8073 85.0
8087 92.0
8109 17.0
8111 6.0
8117 220.0
8125 48.0
9001 7780.0
9009 7807.0
12003 231.0
12007 200.0
12023 473.0
12055 622.0
12057 9829.0
12061 1168.0
12065 115.0
12081 2469.0
12085 664.0
12087 439.0
1209

5035 487.0
5095 103.0
5099 57.0
6039 1718.0
6071 34001.0
6075 3319.0
6077 7468.0
6099 5911.0
12013 102.0
12077 101.0
12079 123.0
12086 30431.0
12093 331.0
12123 210.0
12125 75.0
13019 162.0
13063 2357.0
13095 727.0
13099 119.0
13109 67.0
13215 1354.0
13261 223.0
13267 199.0
13301 88.0
13303 215.0
16065 290.0
20057 120.0
21013 264.0
21045 135.0
21065 110.0
21075 37.0
21109 94.0
21121 327.0
21129 44.0
21203 145.0
24510 2567.0
26085 26.0
28013 122.0
29223 51.0
30035 52.0
31043 97.0
31047 75.0
34011 1608.0
34017 6107.0
34031 3618.0
34039 4475.0
35005 492.0
35019 20.0
35023 31.0
35033 10.0
35045 1048.0
35051 42.0
35053 81.0
36047 1233.0
36081 1163.0
36087 3121.0
37083 597.0
38005 16.0
40001 416.0
40139 175.0
41049 78.0
45009 121.0
45033 480.0
45053 222.0
45075 1114.0
45089 486.0
47007 85.0
47061 83.0
48003 107.0
48013 756.0
48017 24.0
48025 251.0
48069 20.0
48079 10.0
48095 18.0
48101 11.0
48103 39.0
48113 28427.0
48117 64.0
48153 54.0
48169 8.0
48177 326.0
48195 26.0
48201 33996.0
48207 57

In [178]:
for j in location: 
    if j in K:
        COVID_14days[j] = K[j]
        print (j, COVID_14days[j], K[j])

    if j in Q:
        COVID_14days[j] = Q[j]
        print (j, COVID_14days[j])
        
    if j in B:
        COVID_14days[j] = B[j]
        print (j, COVID_14days[j])
        
    if j in M:
        COVID_14days[j] = M[j]
        print (j, COVID_14days[j])

    if j in R:
        COVID_14days[j] = R[j]
        print (j, COVID_14days[j])

36085 3713
36061 6242
36047 13856 13856
36081 13681
36005 8777


In [179]:
print (covid_cases_county_ny_times)

{1001.0: 479.0, 1003.0: 1801.0, 1005.0: 147.0, 1007.0: 172.0, 1009.0: 396.0, 1011.0: 81.0, 1013.0: 99.0, 1015.0: 999.0, 1017.0: 395.0, 1019.0: 114.0, 1021.0: 190.0, 1023.0: 18.0, 1025.0: 347.0, 1027.0: 69.0, 1029.0: 98.0, 1031.0: 462.0, 1033.0: 405.0, 1035.0: 71.0, 1037.0: 55.0, 1039.0: 229.0, 1041.0: 114.0, 1043.0: 555.0, 1045.0: 376.0, 1047.0: 208.0, 1049.0: 495.0, 1051.0: 793.0, 1053.0: 373.0, 1055.0: 959.0, 1057.0: 208.0, 1059.0: 211.0, 1061.0: 199.0, 1063.0: 48.0, 1065.0: 188.0, 1067.0: 162.0, 1069.0: 821.0, 1071.0: 380.0, 1073.0: 5541.0, 1075.0: 64.0, 1077.0: 656.0, 1079.0: 209.0, 1081.0: 1572.0, 1083.0: 721.0, 1085.0: 96.0, 1087.0: 126.0, 1089.0: 2695.0, 1091.0: 136.0, 1093.0: 211.0, 1095.0: 566.0, 1097.0: 2851.0, 1099.0: 136.0, 1101.0: 1809.0, 1103.0: 867.0, 1105.0: 73.0, 1107.0: 109.0, 1109.0: 286.0, 1111.0: 106.0, 1113.0: 475.0, 1115.0: 595.0, 1117.0: 1686.0, 1119.0: 20.0, 1121.0: 649.0, 1123.0: 598.0, 1125.0: 1350.0, 1127.0: 294.0, 1129.0: 97.0, 1131.0: 105.0, 1133.0: 151.0,

In [180]:
#Parameters
pro_c_s = [(i,county_of_states[i]) for i in location ]
cartesian_pro_county_state = gp.tuplelist(pro_c_s)


In [181]:
df = covid_last14_stats
df['fips'] = df['fips'].astype(int)

In [182]:
###############################################################################################
######################## END calculating different types of vulnerabilities ###################

Since we allocate CHW proportional to the county values of certain vulnaribilites within state, we need a few function to help us with the calculations. 



In [183]:
# This function return the value for the state for the given dictionary

# More specifically sum upt the values for the counties of each state

def total_state(dict_1):
    state_dict = {}
    for s in State:
        state_dict [s] = sum(float(dict_1[j]) for (j,s) in cartesian_pro_county_state.select('*', s) if j in dict_1)  
    return state_dict



In [184]:
# Calculte the population per state by summing up the population in each county in the state
State_pop = total_state(population_county)

In [185]:


# This function returns the ratio of the dict value for county and state of the county
def Proportional(county_level, state_level):
    
    prop = {}
       
    for (j,s) in cartesian_pro_county_state:
        if state_level[s] >= 1e-6 and j in county_level:
            prop[j] = (float(county_level[j])/float(state_level[s]))    
        else:
            prop[j] = 0
                
    return prop

In [186]:
print (ACI_total)
print (ACI_total[35039])

{1001: 9804, 1003: 33019, 1005: 7332, 1007: 4860, 1009: 8954, 1011: 3100, 1013: 5004, 1015: 25387, 1017: 7391, 1019: 5162, 1021: 9558, 1023: 3433, 1025: 6557, 1027: 3375, 1029: 2964, 1031: 9652, 1033: 9471, 1035: 2710, 1037: 2234, 1039: 6987, 1041: 3339, 1043: 15529, 1045: 10717, 1047: 13248, 1049: 17983, 1051: 12740, 1053: 6301, 1055: 21800, 1057: 4277, 1059: 6775, 1061: 6484, 1063: 2858, 1065: 4977, 1067: 3485, 1069: 23013, 1071: 11372, 1073: 125514, 1075: 3015, 1077: 15659, 1079: 6971, 1081: 23354, 1083: 15392, 1085: 3457, 1087: 5200, 1089: 48472, 1091: 4803, 1093: 6723, 1095: 21817, 1097: 93203, 1099: 5664, 1101: 52425, 1103: 21849, 1105: 3094, 1107: 4752, 1109: 6901, 1111: 4507, 1113: 13627, 1115: 13737, 1117: 19953, 1119: 4144, 1121: 19136, 1123: 9809, 1125: 34475, 1127: 13645, 1129: 4400, 1131: 4082, 1133: 5781, 2013: 451, 2016: 439, 2020: 51594, 2050: 9063, 2060: 120, 2068: 145, 2070: 2082, 2090: 13016, 2100: 731, 2105: 512, 2110: 4856, 2122: 12129, 2130: 2919, 2150: 2451, 2158

In [187]:
#ACI
ACI_State = total_state(ACI_total)

In [188]:
# Create dicts for the variables of SVI
E_POV = dict(zip(svi_county.FIPS, svi_county.EP_POV))
E_UNEMP = dict(zip(svi_county.FIPS, svi_county.EP_UNEMP))
E_PCI = dict(zip(svi_county.FIPS, svi_county.EP_PCI))
E_NOHSDP = dict(zip(svi_county.FIPS, svi_county.EP_NOHSDP))
E_AGE65 = dict(zip(svi_county.FIPS, svi_county.EP_AGE65))
E_AGE17 = dict(zip(svi_county.FIPS, svi_county.EP_AGE17))
E_DISABL = dict(zip(svi_county.FIPS, svi_county.EP_DISABL))
E_SNGPNT = dict(zip(svi_county.FIPS, svi_county.EP_SNGPNT))
E_MINRTY = dict(zip(svi_county.FIPS, svi_county.EP_MINRTY))
E_LIMENG = dict(zip(svi_county.FIPS, svi_county.EP_LIMENG))
E_MUNIT = dict(zip(svi_county.FIPS, svi_county.EP_MUNIT))
E_MOBILE = dict(zip(svi_county.FIPS, svi_county.EP_MOBILE))
E_CROWD = dict(zip(svi_county.FIPS, svi_county.EP_CROWD))
E_NOVEH = dict(zip(svi_county.FIPS, svi_county.EP_NOVEH))
E_GROUPQ = dict(zip(svi_county.FIPS, svi_county.EP_GROUPQ))


# Calculate the state value for the SVI variables
E_POV_State = total_state(E_POV)
E_UNEMP_State = total_state(E_UNEMP) 
E_PCI_State = total_state(E_PCI)
E_NOHSDP_State = total_state(E_NOHSDP) 
E_AGE65_State = total_state(E_AGE65)
E_AGE17_State = total_state(E_AGE17)
E_DISABL_State = total_state(E_DISABL)
E_SNGPNT_State = total_state(E_SNGPNT)
E_MINRTY_State = total_state(E_MINRTY)
E_LIMENG_State = total_state(E_LIMENG)
E_MUNIT_State = total_state(E_MUNIT)
E_MOBILE_State = total_state(E_MOBILE)
E_CROWD_State = total_state(E_CROWD)
E_NOVEH_State = total_state(E_NOVEH)
E_GROUPQ_State = total_state(E_GROUPQ)



# Calculate the proportinal values for the SVI variables
E_POV_Prop = Proportional(E_POV, E_POV_State )
E_UNEMP_Prop = Proportional(E_UNEMP, E_UNEMP_State ) 
E_PCI_Prop = Proportional(E_PCI, E_PCI_State )
E_NOHSDP_Prop = Proportional(E_NOHSDP, E_NOHSDP_State ) 
E_AGE65_Prop = Proportional(E_AGE65, E_AGE65_State )
E_AGE17_Prop = Proportional(E_AGE17, E_AGE17_State )
E_DISABL_Prop = Proportional(E_DISABL, E_DISABL_State )
E_SNGPNT_Prop = Proportional(E_SNGPNT, E_SNGPNT_State )
E_MINRTY_Prop = Proportional(E_MINRTY, E_MINRTY_State )
E_LIMENG_Prop = Proportional(E_LIMENG, E_LIMENG_State )
E_MUNIT_Prop = Proportional(E_MUNIT, E_MUNIT_State )
E_MOBILE_Prop = Proportional(E_MOBILE, E_MOBILE_State )
E_CROWD_Prop = Proportional(E_CROWD, E_CROWD_State )
E_NOVEH_Prop = Proportional(E_NOVEH, E_NOVEH_State )
E_GROUPQ_Prop = Proportional(E_GROUPQ, E_GROUPQ_State )

ACI_Prop = Proportional(ACI_total, ACI_State)


# SVI calculation 

We calculate the ratio of county value to state value by population for each SVI variables (we use EP-estimate percentage- values in the CDC data set), then we take the average of all 15 SVI variables. 

Let SVI variable set be K, where  

K = { Below Poverty, Unemployed, Income, No High School Diploma, Aged 65 or Older, Aged 17 or Younger, Civilian with a Disability, Single-Parent Households, Minority, Speaks English “Less than Well”, Multi-Unit Structures, Mobile Homes, Crowding, No Vehicle, Group Quarters }

We will use these variables in a county base and state base. While County base values are exactly same as the estimated values for these variables in the CDC website, to calculate the state base, we simply sum the county values for all of the counties in each state for each variable. Then we use the following formula to calculate the SVI value for each county.

Let $S$ is the set of states and $j$ is a county in the state $s$, where $s \in S$, $c^k_j$ SVI variable $k \in K$ value for county j, and $c_s$ SVI variable value for state s.

$SVI_j = \frac{1}{15}\sum_{k \in K} \frac{c^k_j}{c^k_s}$


In [189]:

from collections import Counter
# Sum all SVI variable values for each county
SVI_county_sum = dict(Counter(E_POV_Prop) + Counter(E_UNEMP_Prop) + Counter(E_PCI_Prop) + Counter(E_NOHSDP_Prop) + Counter(E_AGE65_Prop) + Counter(E_AGE17_Prop) + Counter(E_DISABL_Prop) + Counter(E_SNGPNT_Prop) + Counter(E_MINRTY_Prop) + Counter(E_LIMENG_Prop) + Counter(E_MUNIT_Prop) + Counter(E_MOBILE_Prop) + Counter(E_CROWD_Prop) + Counter(E_NOVEH_Prop) + Counter(E_GROUPQ_Prop))

# Divide the sum of all SVI variable values
SVI_county = {j: SVI_county_sum[j]/15 for j in SVI_county_sum }


# Proportional Allocation

We consider allocating 1 million CHW over the states proportional to Medicaid enrollment in each state. Further, we allocate CHW to counties in each state proportional to different county vulnerability criterias as follow.

- MEDICAID
- SVI
- YPLL
- UNEMPLOYMENT
- LAST 14 DAYS COVID CASES
- LAST 14 DAYS COVID CASES / POP
- COVID DEATHS / POP

To calculate the total number of allocated CHW to per county according to these vulnerability criterias, we define the following function called "Proportional_allocation", in which we multiply the CHW allocated to each state with the ratio of the chosen vulnerability criteria of the county to the chosen vulnerability criteria of the state, the function return a dictionary with the counties as keys and the number of CHW allocated to each county for the chosen vulnerability criteria as values. 

In [190]:
def Proportional_allocation(county_level, state_level, state_budget):
    prop_allocate = {}
       
    for (j,s) in cartesian_pro_county_state:
        if state_level[s] >= 1e-6 and j in county_level:
            #print (j,s, county_level[j],state_level[s], state_budget[s])
            prop_allocate[j,s] = (float(county_level[j])/float(state_level[s]))*float(state_budget[s])
        
        else:
            prop_allocate[j,s] = 0
            
    
    return prop_allocate

In [191]:
Medicaid_dem = Proportional_allocation(ACI_total, ACI_State, Medicaid_state)
Medicaid_demand = {m[0]: Medicaid_dem[m] for m in Medicaid_dem}
for s in State:
#    print (s)
#    print ( ACI_State[s])
    print (s,Medicaid_state[s])
#for m in Medicaid_demand:
#    print (m, Medicaid_demand[m])

NEW MEXICO 772102
ALABAMA 957116
ALASKA 231145
ARIZONA 1839932
ARKANSAS 830467
CALIFORNIA 11847711
COLORADO 1337805
CONNECTICUT 874974
DELAWARE 239009
FLORIDA 3892552
GEORGIA 1928703
IDAHO 340742
ILLINOIS 2987496
INDIANA 1602976
IOWA 699741
KANSAS 401103
KENTUCKY 1416013
LOUISIANA 1585024
MAINE 232455
MARYLAND 1372695
MASSACHUSETTS 1616404
MICHIGAN 2439425
MINNESOTA 1085778
MISSISSIPPI 632427
MISSOURI 923641
MONTANA 257006
NEBRASKA 254159
NEVADA 685073
NEW HAMPSHIRE 193436
NEW JERSEY 1759653
NEW YORK 6263164
NORTH CAROLINA 1851558
NORTH DAKOTA 96757
OHIO 2788134
OKLAHOMA 797220
OREGON 1053931
PENNSYLVANIA 3069309
RHODE ISLAND 305208
SOUTH CAROLINA 1048276
SOUTH DAKOTA 114059
TENNESSEE 1489536
TEXAS 4457644
UTAH 338812
VERMONT 161049
VIRGINIA 1497770
WASHINGTON 1790395
WEST VIRGINIA 521290
WISCONSIN 1112844
WYOMING 59302
HAWAII 351337
DISTRICT OF COLUMBIA 250860
PUERTO RICO 1622194


In [192]:
#print ('Med', Medicaid_demand[35039])
#print ('Med', Medicaid_demand[4017], 'Pop', population_county[4017], 'Med_capita', 100000*(Medicaid_demand[4017]/population_county[4017]))

In [193]:
#Further we create additional vulnerability values by considering SVI, YPLL, Unemployment, COVID, COVID_capita, COVID_death and COVID_death capita and the number of Medicaid enrolles in each county together

Covid_capita = {j: 100000*(COVID_14days[j]/population_county[j]) for j in location}

Covid_death_capita = {j: 100000*(County_covid_death[j]/population_county[j]) for j in location}

Medicaid_capita = {j: 100000*(Medicaid_demand[j]/population_county[j]) for j in location}

Unemployment_capita = dict(zip(df_unemp.FIPS, df_unemp.Unemployed_Rate))

#Three_month_lag_covid_death_capita = {j: 100000*(three_month_lag_death[j]/population_county[j]) for j in location}

#Two_month_lag_covid_death_capita = {j: 100000*(two_month_lag_death[j]/population_county[j]) for j in location}

#One_month_lag_covid_death_capita = {j: 100000*(one_month_lag_death[j]/population_county[j]) for j in location}

In [194]:
for j in location:
    print (j, County_covid_death[j], population_county[j], Covid_death_capita[j])

35039 52.0 38921 133.6039670100974
1001 62.0 55869 110.97388533891782
1009 86.0 57826 148.7220281534258
1013 60.0 19448 308.51501439736734
1015 206.0 113605 181.3300470929977
1017 70.0 33254 210.50099236182112
1031 64.0 52342 122.27274464101487
1033 94.0 55241 170.16346554189823
1039 80.0 37049 215.9302545277875
1043 124.0 83768 148.02788654378762
1045 85.0 49172 172.86260473440169
1051 110.0 81209 135.45296703567337
1055 181.0 102268 176.98595846208002
1067 25.0 17205 145.30659691950015
1069 164.0 105882 154.88940518690617
1071 45.0 51626 87.16538178437222
1077 107.0 92729 115.39000744103787
1079 55.0 32924 167.051391082493
1083 81.0 98915 81.88849011777789
1089 217.0 372909 58.191140465904546
1095 123.0 96774 127.1002542005084
1097 569.0 413210 137.70237893565016
1103 142.0 119679 118.65072402008707
1111 36.0 22722 158.43675732770004
1113 15.0 57961 25.87947067855972
1115 130.0 89512 145.23192421127894
1117 132.0 217702 60.63334282643246
1121 112.0 79978 140.03851059041236
2261 2.0 9

31031 6.0 5689 105.4666901037089
31033 15.0 8910 168.35016835016833
31035 11.0 6203 177.33354828308885
31039 9.0 8846 101.74089984173638
31041 17.0 10777 157.7433423030528
31051 8.0 5636 141.94464158978
31053 49.0 36565 134.00793108163546
31059 9.0 5462 164.7748077627243
31065 4.0 4676 85.5431993156544
31067 16.0 21513 74.37363454655325
31075 4.0 623 642.0545746388443
31081 16.0 9324 171.6001716001716
31085 0.0 922 0.0
31089 14.0 10067 139.0682427734181
31093 15.0 6445 232.73855702094647
31095 1.0 7046 14.19244961680386
31099 4.0 6495 61.585835257890686
31103 0.0 806 0.0
31107 7.0 8332 84.01344215074413
31111 39.0 34914 111.70304175975254
31121 11.0 7755 141.84397163120568
31123 15.0 4642 323.1365790607497
31125 10.0 3519 284.1716396703609
31127 12.0 6972 172.1170395869191
31131 3.0 16012 18.735948038970772
31135 11.0 2891 380.4911795226565
31137 8.0 9034 88.55435023245516
31139 20.0 7148 279.79854504756577
31143 9.0 5213 172.64530980241705
31145 9.0 10724 83.92390898918313
31147 15.0 

16013 14.0 23021 60.81403935537118
16015 2.0 7831 25.539522410930914
16019 131.0 119062 110.02670877358015
16027 250.0 229849 108.76706011337879
16029 11.0 7155 153.73864430468203
16041 12.0 13876 86.48025367541078
16043 14.0 13099 106.8783876631804
16051 18.0 29871 60.25911419102139
16059 9.0 8027 112.12158963498193
16061 6.0 3838 156.3314226159458
16071 3.0 4531 66.21054954756124
16079 30.0 12882 232.8830926874709
16081 4.0 12142 32.943501894251355
16083 120.0 86878 138.12472662814523
16087 19.0 10194 186.38414753776732
17005 24.0 16426 146.10982588579083
17009 11.0 6578 167.22408026755852
17017 31.0 12147 255.20704700749155
17025 40.0 13184 303.3980582524272
17033 35.0 18667 187.49665184550275
17037 93.0 104897 88.65839823827183
17041 34.0 19465 174.67248908296943
17043 1168.0 922921 126.55471053318757
17051 53.0 21336 248.40644919385076
17059 4.0 4828 82.8500414250207
17071 16.0 6646 240.74631357207343
17085 25.0 21235 117.73016246762421
17087 15.0 12417 120.80212611741968
17089 70

16031 22.0 24030 91.55222638368706
16035 10.0 8756 114.20740063956144
16037 3.0 4315 69.52491309385863
16049 16.0 16667 95.99808003839922
16057 6.0 40108 14.959609055550017
16067 26.0 21039 123.58001806169496
16075 27.0 23951 112.73015740470126
16077 6.0 7681 78.11482879833355
17069 8.0 3821 209.36927505888508
17097 918.0 696535 131.79524359867057
17109 51.0 29682 171.82130584192439
17143 252.0 179179 140.64148142360432
17165 54.0 23491 229.87527138052872
17201 414.0 282572 146.5113316252141
18097 1398.0 964582 144.93324569606315
18157 139.0 195732 71.01547013263033
19047 24.0 16820 142.6872770511296
19049 72.0 93453 77.04407563160092
19169 38.0 97117 39.128062028275174
20019 3.0 3250 92.3076923076923
20049 0.0 2530 0.0
20061 21.0 31670 66.30880959898958
20069 7.0 5988 116.9004676018704
20093 9.0 3838 234.4971339239187
20161 28.0 74232 37.71958185149262
20189 6.0 5485 109.38924339106654
21015 50.0 133581 37.43047289659458
21053 25.0 10218 244.66627520062633
21069 8.0 14581 54.865921404

6065 2777.0 2470546 112.40430253069566
6069 48.0 62808 76.42338555598013
6073 2344.0 3338330 70.21474809260918
6095 111.0 447643 24.796545461450304
6113 138.0 220500 62.585034013605444
8011 20.0 5577 358.61574323112785
8031 728.0 727211 100.10849670865815
8089 57.0 18278 311.85031185031187
8105 10.0 11267 88.75477056891809
9003 2048.0 891720 229.66850580899836
11001 870.0 705749 123.27328837872956
12011 2018.0 1952778 103.33995979061625
12063 128.0 46414 275.7788598267764
12071 771.0 770577 100.05489393013288
12095 892.0 1393452 64.01368687260128
12097 334.0 375751 88.88865232560926
12133 30.0 25473 117.77175833235191
13001 50.0 18386 271.94604590449256
13009 88.0 44890 196.03475161505904
13025 25.0 19109 130.8284054633942
13027 37.0 15457 239.3737465226111
13033 33.0 22383 147.43331993030426
13055 55.0 24789 221.87260478438017
13059 84.0 128331 65.45573555882834
13087 57.0 26404 215.87638236630815
13089 607.0 759297 79.94236774279366
13091 85.0 20605 412.5212327105072
13097 137.0 1463

8023 2.0 3887 51.45356315924878
12043 12.0 13811 86.88726377525161
12067 23.0 8422 273.09427689384944
13043 38.0 10803 351.75414236786077
13061 3.0 2834 105.85744530698659
13065 16.0 6618 241.76488365064975
13161 38.0 15115 251.4058881905392
13163 48.0 15362 312.4593151933342
13205 67.0 21863 306.45382609888856
13243 31.0 6778 457.36205370315724
13271 44.0 15860 277.42749054224464
13273 37.0 8531 433.7123432188489
20055 67.0 36467 183.72775385965394
20175 28.0 21428 130.67015120403212
21063 1.0 7517 13.303179459890915
21071 18.0 35589 50.5774256090365
21095 35.0 26010 134.56362937331795
21127 8.0 15317 52.22954886727165
21133 5.0 21553 23.198626641302834
21153 1.0 12161 8.223007976317737
21175 0.0 13309 0.0
22029 50.0 19259 259.61887948491614
22041 93.0 20015 464.6515113664752
22097 221.0 82124 269.1052554673421
22107 5.0 4334 115.36686663590216
22117 107.0 46194 231.6318136554531
28069 20.0 9742 205.29665366454526
28125 17.0 4321 393.42744735015043
28151 125.0 43909 284.6796784258353


In [195]:
County_covid_death[9009], population_county[9009], Covid_death_capita[9009]

(1738.0, 854757, 203.33264307867614)

In [196]:
# Dictionary for the total Covid per capita for each state
Total_covid_cap = total_state(Covid_capita) 


# Dictionary for the total Covid death per capita for each state
Total_covid_death_cap = total_state(Covid_death_capita) 


# Dictionary for the total Medicaid per capita for each state
Total_medicaid_cap = total_state(Medicaid_capita)


# Dictionary for the total Unemployment per capita for each state
Total_unemployment_cap = total_state(Unemployment_capita)



In [197]:
# Dictionaries for the different vulnerability criteria values for states

# Dictionary for total Medicaid patient numbers for each state
Medicaid_demand_state = total_state(Medicaid_demand) 

# Dictionary for total positive COVID cases for last 14 days in each state
Covid_state = total_state(COVID_14days) 

# Dictionary for total SVI values for each state
SVI_state = total_state(SVI_county) 

# Dictionary for total YPLL values for each state
YPLL_state = total_state(YPLL) 

# Dictionary for total Unemployment numbers for each state
Unemployment_state = total_state(Unemployment)  

In [198]:
print(Medicaid_demand_state)

{'NEW MEXICO': 772102.0, 'ALABAMA': 957116.0000000001, 'ALASKA': 231145.0, 'ARIZONA': 1839932.0, 'ARKANSAS': 830467.0000000003, 'CALIFORNIA': 11847711.0, 'COLORADO': 1337805.0, 'CONNECTICUT': 874974.0, 'DELAWARE': 239009.0, 'FLORIDA': 3892551.999999999, 'GEORGIA': 1928702.999999999, 'IDAHO': 340742.0000000001, 'ILLINOIS': 2987496.0000000005, 'INDIANA': 1602975.9999999995, 'IOWA': 699741.0, 'KANSAS': 401103.0000000001, 'KENTUCKY': 1416013.0000000002, 'LOUISIANA': 1585023.9999999995, 'MAINE': 232454.99999999994, 'MARYLAND': 1372695.0, 'MASSACHUSETTS': 1616403.9999999998, 'MICHIGAN': 2439425.0000000005, 'MINNESOTA': 1085778.0000000002, 'MISSISSIPPI': 632427.0000000001, 'MISSOURI': 923640.9999999998, 'MONTANA': 257005.99999999994, 'NEBRASKA': 254159.0, 'NEVADA': 685073.0, 'NEW HAMPSHIRE': 193436.0, 'NEW JERSEY': 1759653.0, 'NEW YORK': 6263164.000000002, 'NORTH CAROLINA': 1851557.999999999, 'NORTH DAKOTA': 96756.99999999999, 'OHIO': 2788133.9999999995, 'OKLAHOMA': 797219.9999999998, 'OREGON

# 1 million CHW allocation to states

We allocate 1 million CHWs to states proportional to total Medicaid enrolles in each state.

Let's $FedCHW$ represents the number of CHW will be allocated within states by the federal government, which is 1 million in our project. $TotMed$ represents the total Medicaid enrollee numbers over the US, $Med_s$ is the total Medicaid enrollee numbers in state $s \in S$, and $CHW_s$ is the total number of CHW allocated to state $ s\in S$. 

$CHW_s = FedCHW*\frac{Med_s}{TotMed}$



In [199]:
# We consider allocation of 1 million CHW all over the US
Federal_budget_CHW = 1000000

# First, we calculate the Total Medicaid enrolles all over the US
Total_federal_need = sum(Medicaid_demand_state[s] for s in State)

# Allocate the 1 million CHWs proportional to Medicaid enrolles in each state
Medicaid_budget_state = {s: (Medicaid_demand_state[s]/Total_federal_need)*Federal_budget_CHW  for s in State}

In [200]:
for s in State:
    print (s, Medicaid_demand_state[s], Total_federal_need, Federal_budget_CHW, (Medicaid_demand_state[s]/Total_federal_need)*Federal_budget_CHW)

NEW MEXICO 772102.0 76277412.0 1000000 10122.288889402804
ALABAMA 957116.0000000001 76277412.0 1000000 12547.830018144821
ALASKA 231145.0 76277412.0 1000000 3030.3204309029256
ARIZONA 1839932.0 76277412.0 1000000 24121.583988717393
ARKANSAS 830467.0000000003 76277412.0 1000000 10887.456433367199
CALIFORNIA 11847711.0 76277412.0 1000000 155323.9771690209
COLORADO 1337805.0 76277412.0 1000000 17538.67842291241
CONNECTICUT 874974.0 76277412.0 1000000 11470.945028916292
DELAWARE 239009.0 76277412.0 1000000 3133.417793461582
FLORIDA 3892551.999999999 76277412.0 1000000 51031.516381284666
GEORGIA 1928702.999999999 76277412.0 1000000 25285.37543984842
IDAHO 340742.0000000001 76277412.0 1000000 4467.141596256571
ILLINOIS 2987496.0000000005 76277412.0 1000000 39166.195098491284
INDIANA 1602975.9999999995 76277412.0 1000000 21015.08110946396
IOWA 699741.0 76277412.0 1000000 9173.633211362756
KANSAS 401103.0000000001 76277412.0 1000000 5258.47678209114
KENTUCKY 1416013.0000000002 76277412.0 10000

In [201]:
print (Covid_state)

{'NEW MEXICO': 12420.0, 'ALABAMA': 37567.0, 'ALASKA': 2905.0, 'ARIZONA': 99048.0, 'ARKANSAS': 27742.0, 'CALIFORNIA': 435467.0, 'COLORADO': 23435.0, 'CONNECTICUT': 31804.0, 'DELAWARE': 8623.0, 'FLORIDA': 162781.0, 'GEORGIA': 106249.0, 'IDAHO': 9666.0, 'ILLINOIS': 69803.0, 'INDIANA': 44940.0, 'IOWA': 15787.0, 'KANSAS': 24306.0, 'KENTUCKY': 41336.0, 'LOUISIANA': 33535.0, 'MAINE': 7301.0, 'MARYLAND': 32629.0, 'MASSACHUSETTS': 58673.0, 'MICHIGAN': 43527.0, 'MINNESOTA': 17247.0, 'MISSISSIPPI': 23941.0, 'MISSOURI': 27472.0, 'MONTANA': 5136.0, 'NEBRASKA': 11044.0, 'NEVADA': 21170.0, 'NEW HAMPSHIRE': 9911.0, 'NEW JERSEY': 72927.0, 'NEW YORK': 157295.0, 'NORTH CAROLINA': 91794.0, 'NORTH DAKOTA': 2043.0, 'OHIO': 82777.0, 'OKLAHOMA': 38773.0, 'OREGON': 11919.0, 'PENNSYLVANIA': 79963.0, 'RHODE ISLAND': 12676.0, 'SOUTH CAROLINA': 59264.0, 'SOUTH DAKOTA': 3666.0, 'TENNESSEE': 50090.0, 'TEXAS': 276822.0, 'UTAH': 28801.0, 'VERMONT': 1944.0, 'VIRGINIA': 69897.0, 'WASHINGTON': 27837.0, 'WEST VIRGINIA': 1

# Proportional allocation for different vulnerability values

Let V = {Medicaid, SVI, YPLL, Unemployment, COVID, COVID_capita, COVID_death and COVID_death capita, SVI and Medicaid, YPLL and Medicaid, Unemployment and Medicaid, COVID and Medicaid, COVID_capita and Medicaid, COVID_death and Medicaid and COVID_death capita and Medicaid}. We assume $v_j$ represent the vulnerability value for county $j \in J$, while $v_s$ represent the sum of the vulnerability values for each county in the state of county j.

$Prop_{v_j} = \frac{v_j}{v_s}*CHW_s$



In [202]:
# Calling proportional allocation function for different vulnerability criterias

# Proportional allocation according to cumulative Covid death in per capita in each county
Proportional_to_covid_death_cap = Proportional_allocation(Covid_death_capita, Total_covid_death_cap,Medicaid_budget_state)

# Propportional allocation according to Medicaid enrollee number in each county
Proportional_to_medicaid = Proportional_allocation(Medicaid_demand, Medicaid_demand_state,Medicaid_budget_state )

# Proportional allocation according to Medicaid enrolles per capita in each county
Proportional_to_medicaid_cap = Proportional_allocation(Medicaid_capita, Total_medicaid_cap, Medicaid_budget_state )

# Proportional allocation according to last 14 days positive COVID cases in each county
Proportional_to_covid = Proportional_allocation(COVID_14days, Covid_state, Medicaid_budget_state)

# Proportional allocation according to SVI score in each county
Proportional_to_SVI = Proportional_allocation(SVI_county, SVI_state, Medicaid_budget_state)

# Proportional allocation according to YPLL in each county
Proportional_to_YPLL = Proportional_allocation(YPLL, YPLL_state, Medicaid_budget_state)

# Proportional allocation according to Unemployment  in each county
Proportional_to_unemployment = Proportional_allocation(Unemployment, Unemployment_state, Medicaid_budget_state)

# Proportional allocation according to Medicaid enrolles per capita in each county
Proportional_to_unemployment_cap = Proportional_allocation(Unemployment_capita, Total_unemployment_cap, Medicaid_budget_state )

# Proportional allocation according to last 14 days positive COVID cases per capita in each county
Proportional_to_covid_capita = Proportional_allocation(Covid_capita, Total_covid_cap, Medicaid_budget_state)



# Normalize values for comparison
To be able compare the different vulnerability values for each county, we normalize all vulnerability values as follows. 

Let $m_{v_s} = \min \{v_j, \text{ for county j in state s }\}$  and 
$M_{v_s} = \max \{v_j, \text{ for county j in state s }\}$.

We calculate the normalize value for each vulnerability for each county by substracting the min vulnerability in the state of the county and dividing that by the differences between max and min value of the vulnerability values in the state. Mathematical formulation for the normalization is as follows.

$N_{v_j} = \frac{v_j - m_{v_s}}{M_{v_s} - m_{v_s}}$ 

for each $v \in V$, where V = {Medicaid, SVI, YPLL, Unemployment, COVID, COVID_capita, COVID_death and COVID_death capita, SVI and Medicaid, YPLL and Medicaid, Unemployment and Medicaid, COVID and Medicaid, COVID_capita and Medicaid, COVID_death and Medicaid and COVID_death capita and Medicaid}, j is a county in each state $s \in S$.


In [203]:
#Normalize function to normalize the vulnerability values to be able to compare them

def normalize(dict_1):
    
    result = {}
    min_data = {s: min(dict_1[j] for j in location if (j,s) in cartesian_pro_county_state) for s in State }
    max_data = {s: max(dict_1[j] for j in location if (j,s) in cartesian_pro_county_state) for s in State }
    
    for (j,s) in cartesian_pro_county_state:
        
        if (max_data[s] - min_data[s]) != 0 :
    
            result[j] = (dict_1[j] - min_data[s])/(max_data[s] - min_data[s])
        
        else:
            result[j] = 1
    
    return result 

# Percentile Rank

<a href="https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.percentileofscore.html"> The function scipy.stats.percentileofscore (a, score, kind='rank')   </a>
computes the percentile rank of a score relative to a list of scores. 
"rank": Average percentage ranking of score. In case of multiple matches, average the percentage rankings of all matching scores.

In [204]:
from scipy import stats

# Calculate percentile ranks

def percentile_ranks(data):
    x = {s: [] for s in State}

    for (j,s) in cartesian_pro_county_state:
         
        x[s].append(data[j])
    
    
    
    percentile_ranks = {i: stats.percentileofscore(x[s], data[i], 'rank') for (i,s) in cartesian_pro_county_state}

    return percentile_ranks

In [205]:
# Write timestamp 

time_stamp = time.strftime('%m-%d-%Y %H:%M:%S')
with open('Output/time_stamp.csv','w') as f:
    w = csv.writer(f)
    now = time.strftime('%m/%d/%Y %H:%M:%S')
    w.writerow(['time',now])
    

In [206]:
#print (Medicaid_demand)

In [207]:
#Write a function to order the dicts
def order_k(dict_1):
    dict_2 = {}
    for m in location:
        if m in dict_1.keys():
            dict_2[m] = dict_1[m]
        else:
            dict_2[m] = 0
    
    return dict_2
            

In [208]:


Medicaid_demand = order_k(Medicaid_demand)#{m: Medicaid_demand[m] for m in location}
COVID_14days = order_k(COVID_14days)#{m: COVID_14days[m] for m in location}
SVI_county = order_k(SVI_county)#{m: SVI_county[m] for m in location}
YPLL = order_k(YPLL)#{m: YPLL[m] for m in location}
Unemployment = order_k(Unemployment)#{m: Unemployment[m] for m in location}
Unemployment_capita = order_k(Unemployment_capita)#{m: Unemployment_capita[m] for m in location}

In [209]:
# Write file allocation with each strategies for each county 

Strategies = ["Medicaid_demand", "Medicaid_capita", "Covid", "SVI"
              , "YPLL","Unemployment", "Unemployment_capita", "Covid_capita",  "Covid_death_capita" ]

fieldnames = []  
fieldnames.append('County_FIPS')


SVI_values = {i:SVI_county[i] for i in location}
s_count = 1
for s in Strategies:   
    fieldnames.append('Proportional_allocation_to_' + s)
    fieldnames.append(s)
    fieldnames.append('Percentile_ranks_' + s)

    
        

writefile = 'Output/County_level_proportional_allocation_for_all_policies.csv'
with open( writefile, 'w' ) as f:
    writer = csv.writer(f)                
    writer.writerow(fieldnames)
    for row in zip(location
                   , Proportional_to_medicaid.values(),         Medicaid_demand.values(),      percentile_ranks(Medicaid_demand).values()
                   , Proportional_to_medicaid_cap.values(),     Medicaid_capita.values(),      percentile_ranks(Medicaid_capita).values()
                   , Proportional_to_covid.values(),            COVID_14days.values(),         percentile_ranks(COVID_14days).values()
                   , Proportional_to_SVI.values(),              SVI_county.values(),           percentile_ranks(SVI_county).values()
                   , Proportional_to_YPLL.values(),             YPLL.values(),                 percentile_ranks(YPLL).values()
                   , Proportional_to_unemployment.values(),     Unemployment.values(),         percentile_ranks(Unemployment).values()
                   , Proportional_to_unemployment_cap.values(), Unemployment_capita.values(),  percentile_ranks(Unemployment_capita).values()
                   , Proportional_to_covid_capita.values(),     Covid_capita.values(),         percentile_ranks(Covid_capita).values()
                   , Proportional_to_covid_death_cap.values(),  Covid_death_capita.values(),   percentile_ranks(Covid_death_capita).values() ):                    
       
        writer.writerow(row)

In [210]:
writefile = 'Output/State_level_allocation.csv'

cl = ['State', 'CHW_allocation']
with open( writefile, 'w' ) as f:
    writer = csv.writer(f)                
    writer.writerow(cl)
    for row in zip( State, Medicaid_demand_state.values()):
        writer.writerow(row)